# StepFun JSON Mode 使用指南

本指南将详细介绍如何使用StepFun的JSON Mode功能，让大模型返回可解析的JSON结果，从而更好地与应用程序集成。

## 什么是JSON Mode？

除了应用在用户的日常对话外，阶跃星辰大模型还可以用于和应用自身的逻辑代码进行一些操作，来实现自然语言到应用逻辑的转化。

比如，你可以使用大模型来实现：
- 用户评论内容的情绪判断
- 内容评分功能
- 自然语言到应用逻辑的转化

阶跃星辰为开发者提供了JSON Mode的支持，帮助开发者获取到可以解析的JSON数据，更加容易与系统进行集成。

## 使用方法

在使用JSON Mode时，你需要做三件事：

1. **在System Prompt中定义JSON结构**：放置你预期大模型给出的输出的JSON的结构和说明（推荐使用JSON Schema的结构描述，帮助大模型理解）
2. **设置response_format**：请求时，设置`response_format`为`{"type": "json_object"}`，从而来让大模型返回可解析的JSON结果
3. **解析和验证结果**：解析大模型返回的结果，并验证是否符合预期。符合预期后即可使用在业务系统中对接

## 情感分析示例

以下是一个使用大模型进行评论情感分析的完整示例：

In [ ]:
# 安装必要的依赖
!pip install openai

In [1]:
from openai import OpenAI
import json

# 初始化阶跃星辰Client
import os

# 设置API密钥和文件路径
STEPFUN_KEY = os.getenv("STEPFUN_API_KEY") 

client = OpenAI(
    base_url="https://api.stepfun.com/v1",
    api_key=STEPFUN_KEY
)

In [8]:
# 定义System Prompt，包含JSON结构和分析规则
system_prompt = """
你是一个评论分析师，会根据用户的输入，分析用户输入的情感，并给出评论的情感分析结果

## 分析规则
1. 如果用户的评论中包含了负面情感词汇，那么情感分析结果为 negative
2. 如果用户的评论中包含了正面情感词汇，那么情感分析结果为 positive
3. 如果用户的评论中没有包含正面或者负面情感词汇，那么情感分析结果为 neutral

## 正面情感词汇
- 好
- 棒
- 优秀

## 负面情感词汇
- 差
- 糟糕
- 糟

## 例子
### 例子 1
#### 输入内容
"这个产品真的很好，我很喜欢"
#### 输出内容
{
"emotion": "positive",
"score": 0.9,
"reason": "用户评论中包含了正面情感词汇"
}

### 例子 2
#### 输入内容
"这个产品好垃圾，我很讨厌"
#### 输出内容
{
"emotion": "negative",
"score": 0.9,
"reason": "用户评论中包含了负面情感词汇"
}

## 输入内容
用户的评论文本

## 输出内容
按照如下结构输出JSON结果
class Response:
emotion: str # 用户评论的情感分析结果，可选项为 positive, negative, neutral
score: float # 用户评论的情感分析得分，范围为0到1之间
reason: str # 用户评论的情感分析结果的原因
"""

In [9]:
# 测试情感分析功能
def analyze_sentiment(comment_text):
    """分析评论的情感"""
    try:
        response = client.chat.completions.create(
            model="step-1-8k",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": comment_text}
            ],
            response_format={"type": "json_object"},
        )
        
        # 解析返回的JSON结果
        result = json.loads(response.choices[0].message.content)
        return result
    
    except Exception as e:
        return {"error": str(e)}

# 测试不同的评论
test_comments = [
    "这个产品真的很好，我很喜欢",
    "这个产品好垃圾，我很讨厌",
    "这个产品还行",
    "质量不错，值得推荐！",
    "完全不行，太失望了"
]

# 执行情感分析
print("=== 情感分析结果 ===")
for i, comment in enumerate(test_comments, 1):
    print(f"\n测试 {i}: {comment}")
    result = analyze_sentiment(comment)
    print(f"结果: {json.dumps(result, ensure_ascii=False, indent=2)}")

=== 情感分析结果 ===

测试 1: 这个产品真的很好，我很喜欢
结果: {
  "emotion": "positive",
  "score": 0.9,
  "reason": "用户评论中包含了正面情感词汇"
}

测试 2: 这个产品好垃圾，我很讨厌
结果: {
  "emotion": "negative",
  "score": 0.9,
  "reason": "用户评论中包含了负面情感词汇"
}

测试 3: 这个产品还行
结果: {
  "emotion": "neutral",
  "score": 0.5,
  "reason": "用户评论中没有包含正面或者负面情感词汇"
}

测试 4: 质量不错，值得推荐！
结果: {
  "emotion": "positive",
  "score": 0.9,
  "reason": "用户评论中包含了正面情感词汇"
}

测试 5: 完全不行，太失望了
结果: {
  "emotion": "negative",
  "score": 0.9,
  "reason": "用户评论中包含了负面情感词汇"
}


## 使用JSON Schema优化返回结构

在使用JSON Mode时，有些时候模型会返回一些不符合我们预期的内容。为了帮助模型更好的理解我们的预期，我们可以使用JSON Schema来帮助模型理解我们的预期。

JSON Schema是用于验证JSON数据结构的强大工具，可以帮助我们定义JSON数据的结构，从而帮助模型更好的理解我们的预期。

In [10]:
# 定义JSON Schema示例
url_extraction_schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "用户输入解析",
    "type": "object",
    "properties": {
        "url": {
            "type": "string",
            "description": "用户输入的URL"
        },
        "notes": {
            "type": "string",
            "description": "用户对于输入的URL的评价"
        }
    },
    "required": ["url", "notes"]
}

print("JSON Schema示例:")
print(json.dumps(url_extraction_schema, ensure_ascii=False, indent=2))

JSON Schema示例:
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "用户输入解析",
  "type": "object",
  "properties": {
    "url": {
      "type": "string",
      "description": "用户输入的URL"
    },
    "notes": {
      "type": "string",
      "description": "用户对于输入的URL的评价"
    }
  },
  "required": [
    "url",
    "notes"
  ]
}


In [11]:
# 使用JSON Schema的完整示例
def extract_url_and_notes(input_text):
    """从文本中提取URL和用户评价"""
    
    # 包含JSON Schema的System Prompt
    schema_prompt = f"""
你是一个专业的文本处理工程师，会根据用户的输入，从中提取URL和用户的评价内容，并以JSON的方式返回。

## 约束
- 你会以下方的JSON Schema的格式返回给用户
{json.dumps(url_extraction_schema, ensure_ascii=False)}
"""
    
    try:
        response = client.chat.completions.create(
            model="step-1-8k",
            messages=[
                {"role": "system", "content": schema_prompt},
                {"role": "user", "content": input_text}
            ],
            response_format={"type": "json_object"},
        )
        
        result = json.loads(response.choices[0].message.content)
        return result
    
    except Exception as e:
        return {"error": str(e)}

# 测试URL提取功能
test_inputs = [
    "我在https://www.example.com上看到了一个很有趣的文章，推荐给大家",
    "这个网站https://github.com很好用，代码质量很高",
    "访问了https://stackoverflow.com，找到了问题的答案"
]

print("=== URL提取测试 ===")
for i, text in enumerate(test_inputs, 1):
    print(f"\n测试 {i}: {text}")
    result = extract_url_and_notes(text)
    print(f"提取结果: {json.dumps(result, ensure_ascii=False, indent=2)}")

=== URL提取测试 ===

测试 1: 我在https://www.example.com上看到了一个很有趣的文章，推荐给大家
提取结果: {
  "url": "https://www.example.com",
  "notes": "我在https://www.example.com上看到了一个很有趣的文章，推荐给大家"
}

测试 2: 这个网站https://github.com很好用，代码质量很高
提取结果: {
  "url": "https://github.com",
  "notes": "这个网站很好用，代码质量很高"
}

测试 3: 访问了https://stackoverflow.com，找到了问题的答案
提取结果: {
  "url": "https://stackoverflow.com",
  "notes": "找到了问题的答案"
}


## 注意事项

在使用JSON Mode时，需要注意以下几点：

1. **检查finish_reason**：你需要检查返回结果的`finish_reason`是否为`stop`。如`finish_reason`为`length`，则是因为受到`max_token`的限制导致无法完整返回内容，所返回的Message可能无法被正常解析。

2. **提供示例**：在Prompt中加入输入和输出的范例，可以帮助大模型理解你的场景和需求，给出更加符合预期的输出。

3. **错误处理**：在生产环境中，建议添加适当的错误处理机制。

In [12]:
# 增强版的情感分析函数，包含完整的错误处理
def enhanced_sentiment_analysis(comment_text, max_retries=3):
    """增强版情感分析，包含错误处理和重试机制"""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="step-1-8k",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": comment_text}
                ],
                response_format={"type": "json_object"},
                max_tokens=500  # 限制最大token数
            )
            
            # 检查finish_reason
            finish_reason = response.choices[0].finish_reason
            print(f"Finish reason: {finish_reason}")
            
            if finish_reason == "length":
                print("警告：返回内容被截断，可能无法完整解析")
                continue
            elif finish_reason != "stop":
                print(f"异常结束原因: {finish_reason}")
                continue
            
            # 解析JSON结果
            result = json.loads(response.choices[0].message.content)
            
            # 验证返回结果是否符合预期结构
            required_fields = ["emotion", "score", "reason"]
            if all(field in result for field in required_fields):
                return result
            else:
                print(f"返回结果缺少必要字段: {required_fields}")
                
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
        except Exception as e:
            print(f"请求错误: {e}")
    
    return {"error": "多次重试后仍然失败"}

# 测试增强版函数
test_comment = "这个产品真的很好，我很喜欢"
print(f"测试评论: {test_comment}")
result = enhanced_sentiment_analysis(test_comment)
print(f"最终结果: {json.dumps(result, ensure_ascii=False, indent=2)}")

测试评论: 这个产品真的很好，我很喜欢
Finish reason: stop
最终结果: {
  "emotion": "positive",
  "score": 0.9,
  "reason": "用户评论中包含了正面情感词汇"
}


## 实际应用场景

JSON Mode可以应用于多种场景：

### 1. 内容审核
- 自动检测文本中的敏感内容
- 返回审核结果和风险等级

### 2. 数据提取
- 从非结构化文本中提取结构化数据
- 识别实体、关系和属性

### 3. 智能分类
- 对文本进行多维度分类
- 返回分类结果和置信度

### 4. 问答系统
- 从文档中提取答案
- 返回结构化的问答结果

### 5. 工作流自动化
- 将自然语言指令转换为结构化操作
- 生成可执行的操作序列

In [ ]:
# 创建一个通用的JSON Mode调用函数
def call_stepfun_json_mode(system_prompt, user_input, model="step-1-8k", max_tokens=500):
    """
    通用的StepFun JSON Mode调用函数
    
    Args:
        system_prompt: 系统提示词
        user_input: 用户输入
        model: 使用的模型
        max_tokens: 最大token数
    
    Returns:
        dict: 解析后的JSON结果或错误信息
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            response_format={"type": "json_object"},
            max_tokens=max_tokens
        )
        
        finish_reason = response.choices[0].finish_reason
        
        if finish_reason == "stop":
            result = json.loads(response.choices[0].message.content)
            return {
                "success": True,
                "data": result,
                "finish_reason": finish_reason
            }
        else:
            return {
                "success": False,
                "error": f"异常结束原因: {finish_reason}",
                "finish_reason": finish_reason
            }
            
    except json.JSONDecodeError as e:
        return {
            "success": False,
            "error": f"JSON解析错误: {str(e)}"
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"请求错误: {str(e)}"
        }

# 示例：使用通用函数进行情感分析
print("=== 使用通用函数进行情感分析 ===")
result = call_stepfun_json_mode(system_prompt, "这个产品太棒了！")
print(json.dumps(result, ensure_ascii=False, indent=2))

=== 使用通用函数进行情感分析 ===
{
  "success": true,
  "data": {
    "emotion": "positive",
    "score": 0.9,
    "reason": "用户评论中包含了正面情感词汇"
  },
  "finish_reason": "stop"
}


## 总结

通过本指南，您已经学会了：

1. **JSON Mode的基本概念**：了解如何使用JSON Mode让大模型返回结构化的JSON结果
2. **三步使用方法**：
   - 在System Prompt中定义JSON结构
   - 设置response_format为json_object
   - 解析和验证返回结果
3. **实际代码示例**：情感分析和URL提取的完整实现
4. **JSON Schema的使用**：如何通过Schema帮助模型更好地理解预期结构
5. **最佳实践**：错误处理、重试机制和结果验证

通过这些技术，您可以更好地将大模型集成到应用程序中，实现自然语言到应用逻辑的无缝转换。